# Part of Lab-Sheet 9 (COM3502-4502-6502 Speech Processing)

##  LPC Analysis

This lab sheet is part of the lecture COM[3502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/level3/com3502.html "Open web page for COM3502 module")-[4502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/level4/com4502.html "Open web page for COM4502 module")-[6502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/msc/com6502.html "Open web page for COM4502 module") Speech Processing at the [University of Sheffield](https://www.sheffield.ac.uk/ "Open web page of The University of Sheffield"), Dept. of [Computer Science](https://www.sheffield.ac.uk/dcs "Open web page of Department of Computer Science, University of Sheffield").

It is probably easiest to open this Jupyter Notebook with [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb#recent=true "Open in Google Colab") since GitHub's Viewer does not always show all details correctly. <a href="https://colab.research.google.com/github/sap-shef/SpeechProcesssingLab/blob/main/Lab-Sheets/LPC/lpc_analysis.ipynb"><img align="right" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open Notebook in Google Colab" title="Open and Execute the Notebook directly in Google Colaboratory"></a>

Please put questions, comments and correction suggestions in the [Blackboard](https://vle.shef.ac.uk) discussion board or send an email to [s.goetze@sheffield.ac.uk](mailto:s.goetze@sheffield.ac.uk).

## Navigation

![LPC Analysis is needed for LPC](images/flow_lpc_analysis.svg)

This Lab Sheet contains several Notebooks (the blue-ish colour in the picture above indicates **this** Notebook to help with navigation):

* The Notebooks [`correlation.ipynb`](./correlation.ipynb),  [`autocorrelation.ipynb`](./LPC/autocorrelation.ipynb) and [`fundamental_frequency.ipynb`](./fundamental_frequency.ipynb)  repeat knowlege already known from lab sheet [Lab-Sheet 3](Lab-Sheet-3.ipynb) and [Lab-Sheet 7](Lab-Sheet-7.ipynb), respectively.
* The Notebook [`lpc_analysis.ipynb`](./lpc_analysis.ipynb)
* The Notebook [`lpc_synthesis.ipynb`](./lpc_synthesis.ipynb)
* The Notebook [`lpc.ipynb`](./lpc.ipynb) is the main Notebook relying an all other Notebooks and the code therein.

## Intended Learning Outcomes (ILOs)

After completing this notebook, students should be able to

- *explain* the idea of linear prediction and
- *translate* the given description of the Levinson-Durbin algorithm into code.

## Introduction

The idea of Linear Prediction Analysis is that we can predict a sample of a speech signal by a weighted combination of a finite number $P$ of preceding samples.
Mathematically we define our predicted sample as
$$\hat{s}[k] = a_1 s[k-1] + a_2 s[k-2] + \ldots + a_P s[k-P] = \sum_{p=1}^P a_p s[k-p]$$
and the prediction error $e[k]$ for each index $k$ is obtained by 
$$e[k] = s[k] - \hat{s}[k] = s[k] - \sum_{p=1}^P a_p s[k-p].$$

The prediction coefficients $a_p$ are then chosen to minimize the sum-squared prediction error
$$E_p = \sum_k e[k]^2.$$
The effect of this is that the resulting spectrum is flattened which we will be able to observe at the end of the [LPC notebook](./lpc.ipynb#Example).
One way to find the coefficients to minimized $E_p$ is to use the Levinson-Durbin algorithm in the next section.

## Levinson-Durbin Algorithm
According to [BCH07] the Levinson-Durbin algorithm to compute $P$ Linear Prediction coefficients $\{a_1,\ldots,a_P\}$ can be formulated like this:

<div style="border: 1px solid #999; padding: 20px; margin: 20px; background: #eee;">
    
**Levinson-Durbin Algorithm**

1. $E_0 = r(0)$

2. **For** $1 \leq l \leq P$:

	1. $\kappa_l = \frac{1}{E_{l-1}}\left[r(l)-\mathbf{a}_{l-1}^T\mathbf{r}_{l-1}^F\right]$

	2. $\mathbf{a}_l = \begin{pmatrix} \mathbf{a}_{l-1}\\0\end{pmatrix} - \kappa_l\begin{pmatrix}-1 \\ \mathbf{a}_{l-1}\end{pmatrix}^F$

	3. $E_l = E_{l-1}(1-\kappa_l^2)$
</div>
    
*where*

- $r(i)$ is the $i$th non-normalized correlation value, $r(i)=\sum_{k=1}^{L-i} s[k]s[k+i]$
- $\mathbf{r}_l$ is the vector $\begin{pmatrix}r(1)\\r(2)\\\vdots\\r(l)\end{pmatrix}$ containing the first $l$ correlations values without $r(0)$
- $\mathbf{a}_l$ is the vector $\begin{pmatrix}a(1)\\a(2)\\\vdots\\a(l)\end{pmatrix}$ containing the first $l$ prediction coefficients
- $(.)^T$ is the transpose of the vector
- $\begin{pmatrix}a_1\\a_2\\\vdots\\a_n\end{pmatrix}^F = \begin{pmatrix}a_n\\a_{n-1}\\\vdots\\a_1\end{pmatrix}$, i.e., it flips the order of the entries in the vector
- $E_l$ is the prediction-error power
- $\kappa_l$ is a so-called *reflection coefficient*; we will return these values and we will later use them to visualize a model of the vocal tract

The coefficients $a_p$, $1\leq p\leq P$ will then be used in an *all-zero* filter
$$\frac{E[z]}{S[z]} = 1 - \sum_{p=1}^P a_pz^{-p}.$$

We will use [scipy.signal.lfilter(a, b, x)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.lfilter.html) for filtering which takes two coefficient arrays $a$ (with entries $a_0$ to $a_P$) and $b$ (with entries $b_0$ to $b_Q$) to filter the signal $x$ to obtain output $y$. The filter works as follows (in the z-domain):
$$\frac{Y[z]}{X[z]} = \frac{a_0 + a_1z^{-1} + \ldots + a_Pz^{-P}}{b_0 + b_1z^{-1} + \ldots + b_Qz^{-Q}}$$

*Note:* In the actual function the names of $a$ and $b$ are swapped which would only lead to confusion here.

To obtain the filter as above we need to obtain a denominator of $1$; we achieve this by seting $b_0 = 1$ and $Q = 0$.
For the desired denominator $1 - \sum_{p=1}^P a_pz^{-p}$ we need to invert all coefficients $a_p$, $1\leq p\leq P$, obtained by the Levinson-Durbin algorithm and additionally set $a_0=1$.
We will do these two steps already in the `levinson` function.

The *reflection coefficients* $\kappa_l$ 

In [2]:
import numpy as np
from scipy.signal import lfilter

In [3]:
def levinson(signal, P):
    # -----------
    # Preparation
    # -----------

    # Compute the non-normalized (just sum the products) autocorrelation for time shifts by 0,...,P samples
    r = np.zeros(P + 1)
    r[0] = np.dot(signal, signal)
    for p in range(1, P + 1):
        r[p] = np.dot(signal[:-p], signal[p:])

    # The prediction error powers
    error = np.zeros(P + 1)
    # The reflection coefficients
    kappa = np.zeros(P)
    # The prediction coefficients
    a = np.zeros(P + 1)

    # a[0] has to be 1 but as we will invert all values in a[] before returning them, we set a[0] = -1
    a[0] = -1
    
    # -------------------------
    # Levinson-Durbin-Algorithm
    # -------------------------

    # 1. Initialization
    error[0] = r[0]

    # 2. Loop through the coefficients and calculate the reflection coefficient, prediction coefficient and prediction error power
    for l in range(1, P + 1):
        # A. Update kappa_l (we use kappa[l-1] for kappa_l as numpy arrays are zero-based)
        kappa[l - 1] = (r[l] - np.dot(a[1:l], np.flip(r[1:l]))) / error[l - 1]
        # B. Update a_l, i.e., all entries a[1], a[2], ..., a[l]
        a[1 : l + 1] = a[1 : l + 1] - kappa[l - 1] * np.concatenate(
            (np.flip(a[1:l]), [-1])
        )
        # C. Update E_l
        error[l] = error[l - 1] * (1 - kappa[l - 1] * kappa[l - 1])

    # Return the inverted values
    return -a, -kappa

## LPC Analysis

Now that we are able to extract the prediction coefficients we want to filter our signal with these coefficients.
This will result in the error signal $E[z]$ which we will return together with the prediction and reflection coefficients.

The LPC analysis takes the following parameters
- `signal`: the signal to analyze,
- `P`: the number of LPC coefficients to extract

and returns a triple with
- `a`: the prediction coefficients as filter coefficients,
- `kappa`: the reflection coefficients, and
- `excitation_signal`: the signal which remains after filtering the signal with the prediction coefficients.

In [1]:
def lpc_analysis(signal, P):
    # Obtain LP parameters from signal
    a, kappa = levinson(signal, P)
    # Extract excitation signal from signal with all zeros filter A
    excitation_signal = lfilter(a, [1], signal)

    return a, kappa, excitation_signal

## Copyright
This notebook is licensed to be used during the lecture COM[3502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/level3/com3502.html "Open web page for COM3502 module")-[4502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/level4/com4502.html "Open web page for COM4502 module")-[6502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/msc/com6502.html "Open web page for COM4502 module") Speech Processing at the [University of Sheffield](https://www.sheffield.ac.uk/ "Open web page of The University of Sheffield"), Dept. of [Computer Science](https://www.sheffield.ac.uk/dcs "Open web page of Department of Computer Science, University of Sheffield"). Any further use (beyond use for the lecture) is only permitted if agreed with the [module lead](mailto:s.goetze@sheffield.ac.uk). 
### Contributors
[Stefan Goetze](http://www.stefan-goetze.de "Web page of Stefan Goetze"),
[Lena Strobl](https://github.com/sleyna/)

## References
<a id='idReferences'></a>

* [BCH07] J. Benesty, J. Chen, Y. Huang: "Linear Prediction", in "Springer Handbook of Speech Processing", chapter 7, pp. 121-134, Springer, Berlin, London, ISBN: [9783540491279](https://www.springer.com/gp/book/9783662533000 "Link to book at springer.com"), 2007, [Online available with login](https://r1.vlereader.com/Reader?ean=9783540491279)